In [1]:
import pandas as pd

In [58]:
df = pd.read_csv('./news_data.csv' ,index_col=0)
nt = pd.read_csv('./nation.csv')
nt1 = nt['list'].tolist()


In [59]:
# colums (title, content, wirteda_at, sub_category)


,title,writer,content,writed_at,url,main_category,sub_category,platform
news_id,,,,,,,,
1128869,"쿠데타 2주년 맞은 미얀마 군정, 국가비상사태 연장···“특수 상황 끝나지 않아”",선명수 기자,쿠데타 2년을 맞은 미얀마 군사정권이 국가비상사태를 연장해 군정 강화에 나섰다. 미...,2023-02-01 23:27:00,NaN,세계,아시아/호주,네이버
1128871,"약속 깬 미얀마 군정, 국가비상사태 6개월 연장",김철오 기자,미얀마 군사정권이 국가비상사태를 연장했다. 쿠데타 수장인 민 아웅 흘라잉 최고사령관...,2023-02-01 23:24:00,NaN,세계,아시아/호주,네이버
1128873,[뉴욕 개장] 연준 금리 발표 앞두고 하락 출발,최서윤 기자,(서울=뉴스1) 최서윤 기자 = 미 중앙은행(FRB·연준)의 금리 인상 발표를 앞둔...,2023-02-01 23:44:00,NaN,세계,미국/중남미,네이버
1129523,"'세계 최대' 노르웨이 국부펀드, 지난해 200조원 손실 기록",이종희 기자,[서울=뉴시스] 이종희 기자 = 세계 최대 규모인 노르웨이 국부펀드가 우크라이나 전...,2023-02-01 16:48:00,NaN,세계,유럽,네이버
1129524,노벨평화상 후보 마감…'反푸틴·우크라전' 대거 포진,신정원 기자,[서울=뉴시스]신정원 기자 = 올해 노벨평화상 후보에도 우크라이나 전쟁과 관련한 반...,2023-02-01 16:57:00,NaN,세계,유럽,네이버
...,...,...,...,...,...,...,...,...
1274434,런던 리버버스 탑승한 오세훈 서울시장,NaN,(런던=연합뉴스) 오세훈 서울시장이 13일(현지시간) 영국 런던 템스강에서 런던의 ...,2023-03-14 10:05:00,NaN,세계,유럽,네이버
1274435,"러 ""한국 등과 항공운항 재개 추진""…작년 3월부터 중단",유철종 기자,(서울=연합뉴스) 유철종 기자 = 러시아 항공 당국이 지난해 초부터 중단된 한국과의...,2023-03-14 10:14:00,NaN,세계,유럽,네이버
1274466,뼛속까지 태우는 '소이탄 비' 우크라 주택가에 쏟아졌다,박지현 기자,[파이낸셜뉴스] 러시아·우크라이나 전쟁이 끝없이 이어지고 있는 가운데 지난 11일(...,2023-03-14 10:44:00,NaN,세계,유럽,네이버
